## Chatbot Tương Tác Phi 3 Mini 4K Instruct với Whisper

### Giới thiệu:
Chatbot Tương Tác Phi 3 Mini 4K Instruct là một công cụ cho phép người dùng tương tác với bản demo Microsoft Phi 3 Mini 4K instruct thông qua đầu vào văn bản hoặc âm thanh. Chatbot này có thể được sử dụng cho nhiều nhiệm vụ khác nhau, chẳng hạn như dịch thuật, cập nhật thời tiết, và thu thập thông tin chung.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Tạo mã truy cập Huggingface của bạn

Tạo một mã mới  
Đặt một tên mới  
Chọn quyền ghi  
Sao chép mã và lưu nó ở nơi an toàn


Tệp Python này thực hiện hai nhiệm vụ chính: nhập mô-đun `os` và thiết lập một biến môi trường.

1. Nhập mô-đun `os`:
   - Mô-đun `os` trong Python cung cấp cách tương tác với hệ điều hành. Nó cho phép bạn thực hiện các tác vụ liên quan đến hệ điều hành, chẳng hạn như truy cập các biến môi trường, làm việc với tệp và thư mục, v.v.
   - Trong đoạn mã này, mô-đun `os` được nhập bằng câu lệnh `import`. Câu lệnh này làm cho các chức năng của mô-đun `os` có thể sử dụng trong tập lệnh Python hiện tại.

2. Thiết lập một biến môi trường:
   - Biến môi trường là một giá trị có thể được truy cập bởi các chương trình chạy trên hệ điều hành. Đây là cách lưu trữ các thiết lập cấu hình hoặc thông tin khác có thể được sử dụng bởi nhiều chương trình.
   - Trong đoạn mã này, một biến môi trường mới được thiết lập bằng cách sử dụng từ điển `os.environ`. Khóa của từ điển là `'HF_TOKEN'`, và giá trị được gán từ biến `HUGGINGFACE_TOKEN`.
   - Biến `HUGGINGFACE_TOKEN` được định nghĩa ngay phía trên đoạn mã này, và nó được gán một giá trị chuỗi `"hf_**************"` bằng cú pháp `#@param`. Cú pháp này thường được sử dụng trong Jupyter notebooks để cho phép người dùng nhập liệu và cấu hình tham số trực tiếp trong giao diện notebook.
   - Bằng cách thiết lập biến môi trường `'HF_TOKEN'`, nó có thể được truy cập bởi các phần khác của chương trình hoặc các chương trình khác chạy trên cùng hệ điều hành.

Tóm lại, đoạn mã này nhập mô-đun `os` và thiết lập một biến môi trường có tên `'HF_TOKEN'` với giá trị được cung cấp trong biến `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Đoạn mã này định nghĩa một hàm có tên là `clear_output`, được sử dụng để xóa nội dung hiển thị của ô hiện tại trong Jupyter Notebook hoặc IPython. Hãy cùng phân tích mã và hiểu rõ chức năng của nó:

Hàm `clear_output` nhận một tham số gọi là `wait`, đây là một giá trị boolean. Theo mặc định, `wait` được đặt là `False`. Tham số này xác định liệu hàm có nên chờ cho đến khi có nội dung mới để thay thế nội dung hiện tại trước khi xóa hay không.

Bản thân hàm này được sử dụng để xóa nội dung hiển thị của ô hiện tại. Trong Jupyter Notebook hoặc IPython, khi một ô tạo ra nội dung hiển thị, chẳng hạn như văn bản được in ra hoặc biểu đồ, nội dung đó sẽ được hiển thị bên dưới ô. Hàm `clear_output` cho phép bạn xóa nội dung hiển thị đó.

Việc triển khai hàm không được cung cấp trong đoạn mã, như được biểu thị bằng dấu ba chấm (...). Dấu ba chấm đại diện cho một chỗ trống dành cho mã thực tế thực hiện việc xóa nội dung hiển thị. Việc triển khai hàm có thể liên quan đến việc tương tác với API của Jupyter Notebook hoặc IPython để loại bỏ nội dung hiện có từ ô.

Tóm lại, hàm này cung cấp một cách tiện lợi để xóa nội dung hiển thị của ô hiện tại trong Jupyter Notebook hoặc IPython, giúp dễ dàng quản lý và cập nhật nội dung hiển thị trong các phiên làm việc mã hóa tương tác.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Sử dụng dịch vụ Edge TTS để thực hiện chuyển văn bản thành giọng nói (TTS). Hãy cùng tìm hiểu từng chức năng được triển khai:

1. `calculate_rate_string(input_value)`: Hàm này nhận một giá trị đầu vào và tính toán chuỗi tốc độ cho giọng nói TTS. Giá trị đầu vào đại diện cho tốc độ mong muốn của giọng nói, trong đó giá trị 1 là tốc độ bình thường. Hàm tính toán chuỗi tốc độ bằng cách trừ 1 từ giá trị đầu vào, nhân với 100, và sau đó xác định dấu dựa trên việc giá trị đầu vào có lớn hơn hoặc bằng 1 hay không. Hàm trả về chuỗi tốc độ theo định dạng "{sign}{rate}".

2. `make_chunks(input_text, language)`: Hàm này nhận văn bản đầu vào và ngôn ngữ làm tham số. Nó chia văn bản đầu vào thành các đoạn nhỏ dựa trên quy tắc ngôn ngữ cụ thể. Trong triển khai này, nếu ngôn ngữ là "English", hàm sẽ chia văn bản tại mỗi dấu chấm (".") và loại bỏ khoảng trắng ở đầu hoặc cuối. Sau đó, nó thêm dấu chấm vào mỗi đoạn và trả về danh sách các đoạn đã lọc.

3. `tts_file_name(text)`: Hàm này tạo tên tệp cho tệp âm thanh TTS dựa trên văn bản đầu vào. Nó thực hiện một số chuyển đổi trên văn bản: loại bỏ dấu chấm ở cuối (nếu có), chuyển văn bản thành chữ thường, loại bỏ khoảng trắng ở đầu và cuối, và thay thế khoảng trắng bằng dấu gạch dưới. Sau đó, nó cắt ngắn văn bản đến tối đa 25 ký tự (nếu dài hơn) hoặc sử dụng toàn bộ văn bản nếu nó rỗng. Cuối cùng, nó tạo một chuỗi ngẫu nhiên bằng cách sử dụng module [`uuid`] và kết hợp với văn bản đã cắt ngắn để tạo tên tệp theo định dạng "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Hàm này hợp nhất nhiều tệp âm thanh thành một tệp âm thanh duy nhất. Nó nhận danh sách các đường dẫn tệp âm thanh và đường dẫn đầu ra làm tham số. Hàm khởi tạo một đối tượng `AudioSegment` rỗng gọi là [`merged_audio`]. Sau đó, nó lặp qua từng đường dẫn tệp âm thanh, tải tệp âm thanh bằng phương pháp `AudioSegment.from_file()` từ thư viện `pydub`, và thêm tệp âm thanh hiện tại vào đối tượng [`merged_audio`]. Cuối cùng, nó xuất tệp âm thanh đã hợp nhất đến đường dẫn đầu ra được chỉ định theo định dạng MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Hàm này thực hiện thao tác TTS bằng dịch vụ Edge TTS. Nó nhận danh sách các đoạn văn bản, tốc độ giọng nói, tên giọng nói, và đường dẫn lưu làm tham số. Nếu số lượng đoạn lớn hơn 1, hàm tạo một thư mục để lưu trữ các tệp âm thanh của từng đoạn. Sau đó, nó lặp qua từng đoạn, xây dựng lệnh Edge TTS bằng cách sử dụng hàm `calculate_rate_string()`, tên giọng nói, và văn bản đoạn, rồi thực thi lệnh bằng hàm `os.system()`. Nếu lệnh thực thi thành công, nó thêm đường dẫn của tệp âm thanh được tạo vào danh sách. Sau khi xử lý tất cả các đoạn, nó hợp nhất các tệp âm thanh riêng lẻ bằng hàm `merge_audio_files()` và lưu tệp âm thanh đã hợp nhất vào đường dẫn lưu được chỉ định. Nếu chỉ có một đoạn, nó trực tiếp tạo lệnh Edge TTS và lưu âm thanh vào đường dẫn lưu. Cuối cùng, nó trả về đường dẫn lưu của tệp âm thanh được tạo.

6. `random_audio_name_generate()`: Hàm này tạo tên tệp âm thanh ngẫu nhiên bằng cách sử dụng module [`uuid`]. Nó tạo một UUID ngẫu nhiên, chuyển đổi thành chuỗi, lấy 8 ký tự đầu tiên, thêm phần mở rộng ".mp3", và trả về tên tệp âm thanh ngẫu nhiên.

7. `talk(input_text)`: Hàm này là điểm khởi đầu chính để thực hiện thao tác TTS. Nó nhận văn bản đầu vào làm tham số. Đầu tiên, nó kiểm tra độ dài của văn bản đầu vào để xác định xem đó có phải là câu dài (lớn hơn hoặc bằng 600 ký tự) hay không. Dựa trên độ dài và giá trị của biến `translate_text_flag`, nó xác định ngôn ngữ và tạo danh sách các đoạn văn bản bằng hàm `make_chunks()`. Sau đó, nó tạo đường dẫn lưu cho tệp âm thanh bằng hàm `random_audio_name_generate()`. Cuối cùng, nó gọi hàm `edge_free_tts()` để thực hiện thao tác TTS và trả về đường dẫn lưu của tệp âm thanh được tạo.

Tóm lại, các hàm này phối hợp với nhau để chia văn bản đầu vào thành các đoạn, tạo tên tệp cho tệp âm thanh, thực hiện thao tác TTS bằng dịch vụ Edge TTS, và hợp nhất các tệp âm thanh riêng lẻ thành một tệp âm thanh duy nhất.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Việc triển khai hai hàm: convert_to_text và run_text_prompt, cũng như khai báo hai lớp: str và Audio.

Hàm convert_to_text nhận đầu vào là audio_path và chuyển đổi âm thanh thành văn bản bằng cách sử dụng một mô hình gọi là whisper_model. Hàm này đầu tiên kiểm tra xem cờ gpu có được đặt thành True hay không. Nếu đúng, whisper_model được sử dụng với các tham số như word_timestamps=True, fp16=True, language='English', và task='translate'. Nếu cờ gpu là False, whisper_model được sử dụng với fp16=False. Kết quả chuyển đổi sau đó được lưu vào một tệp có tên 'scan.txt' và trả về dưới dạng văn bản.

Hàm run_text_prompt nhận đầu vào là một message và một chat_history. Nó sử dụng hàm phi_demo để tạo phản hồi từ chatbot dựa trên message đầu vào. Phản hồi được tạo ra sau đó được truyền vào hàm talk, hàm này chuyển đổi phản hồi thành một tệp âm thanh và trả về đường dẫn của tệp. Lớp Audio được sử dụng để hiển thị và phát tệp âm thanh. Âm thanh được hiển thị bằng cách sử dụng hàm display từ module IPython.display, và đối tượng Audio được tạo với tham số autoplay=True, để âm thanh tự động phát. chat_history được cập nhật với message đầu vào và phản hồi được tạo ra, sau đó trả về một chuỗi rỗng và chat_history đã được cập nhật.

Lớp str là một lớp tích hợp sẵn trong Python, đại diện cho một chuỗi ký tự. Nó cung cấp nhiều phương thức để thao tác và làm việc với chuỗi, chẳng hạn như capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, và nhiều hơn nữa. Các phương thức này cho phép bạn thực hiện các thao tác như tìm kiếm, thay thế, định dạng, và thao tác với chuỗi.

Lớp Audio là một lớp tùy chỉnh đại diện cho một đối tượng âm thanh. Nó được sử dụng để tạo trình phát âm thanh trong môi trường Jupyter Notebook. Lớp này chấp nhận nhiều tham số như data, filename, url, embed, rate, autoplay, và normalize. Tham số data có thể là một mảng numpy, một danh sách mẫu, một chuỗi đại diện cho tên tệp hoặc URL, hoặc dữ liệu PCM thô. Tham số filename được sử dụng để chỉ định một tệp cục bộ để tải dữ liệu âm thanh, và tham số url được sử dụng để chỉ định một URL để tải dữ liệu âm thanh. Tham số embed xác định liệu dữ liệu âm thanh có nên được nhúng bằng URI dữ liệu hay tham chiếu từ nguồn gốc ban đầu. Tham số rate chỉ định tốc độ lấy mẫu của dữ liệu âm thanh. Tham số autoplay xác định liệu âm thanh có nên tự động phát hay không. Tham số normalize chỉ định liệu dữ liệu âm thanh có nên được chuẩn hóa (tái định tỷ lệ) đến phạm vi tối đa có thể hay không. Lớp Audio cũng cung cấp các phương thức như reload để tải lại dữ liệu âm thanh từ tệp hoặc URL, và các thuộc tính như src_attr, autoplay_attr, và element_id_attr để truy xuất các thuộc tính tương ứng cho phần tử âm thanh trong HTML.

Tóm lại, các hàm và lớp này được sử dụng để chuyển đổi âm thanh thành văn bản, tạo phản hồi âm thanh từ chatbot, và hiển thị cũng như phát âm thanh trong môi trường Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
